In [1]:
# Import the libraries
import pandas as pd
import cx_Oracle
from datetime import datetime
import numpy as np

# # Display setting for table
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)


# Create the connection to TFM4CEBERUS database
connection = cx_Oracle.connect('TFM4CEBERUS', 'TFM4CEBERUS', 'OME.BTH.INFINEON.COM')


# Import the EQ status data from V_TFM_RAW_DATA_METAKEY_ITFML table
cur_vis = connection.cursor()
cur_vis.execute("select * from V_TFM_RAW_DATA_METAKEY_ITFM where eq_id in ('WBA124') order by timestamp_start")

data = []

for row in cur_vis: 
    data.append(row)
    

status = pd.DataFrame(data, columns = ['EQ_ID',
                                   'NAME',
                                   'EQ_TYPE_IDENT',
                                   'TIMESTAMP_START',
                                   'TIMESTAMP_END',
                                   'DURATION',
                                   'TR25_3_STATUS',
                                   'TR25_4_STATUS',
                                   'TR25_5_STATUS',
                                   'EQ_STATUS',
                                   'STATE_NAME',
                                   'LEVEL5_NAME',
                                   'LEVEL5',
                                   'LEVEL4_NAME',
                                   'LEVEL4',
                                   'LEVEL3_NAME',
                                   'LEVEL3',
                                   'DEVICE',
                                   'PACKAGE',
                                   'LOT',
                                   'PRODUCT',
                                   'OPERATION'])   

In [2]:
status

,EQ_ID,NAME,EQ_TYPE_IDENT,TIMESTAMP_START,TIMESTAMP_END,DURATION,TR25_3_STATUS,TR25_4_STATUS,TR25_5_STATUS,EQ_STATUS,STATE_NAME,LEVEL5_NAME,LEVEL5,LEVEL4_NAME,LEVEL4,LEVEL3_NAME,LEVEL3,DEVICE,PACKAGE,LOT,PRODUCT,OPERATION
0,WBA124,WBA124,461,2021-01-27 08:11:14,2021-01-27 08:11:19,5.0,3004,4005,5030,14248,PC Buyoff,Production Test,SDT T,Test (SDT),SDT,Scheduled Down (SD),SDT,None,None,None,None,None
1,WBA124,WBA124,461,2021-01-27 08:11:19,2021-01-27 17:11:29,32410.0,3005,4008,5025,14260,PC Buyoff Failed,Machine Failure,UDR MF,Repair (UDR),UDR,Unscheduled Down (UD),UDT,None,None,None,None,None
2,WBA124,WBA124,461,2021-01-27 17:11:29,2021-01-27 17:17:48,379.0,3004,4006,5035,12342,Change Device,Change Setup,SDS CHS,Setup (SDS),SDS,Scheduled Down (SD),SDT,None,None,None,None,None
3,WBA124,WBA124,461,2021-01-27 17:17:48,2021-01-27 20:04:16,9988.0,3002,4002,5011,12379,PlanIdle,No Material,SB M,Standby (SB),SB,Standby (SB),SBY,None,None,None,None,None
4,WBA124,WBA124,461,2021-01-27 20:04:16,2021-01-28 11:05:45,54089.0,3005,4008,5025,14260,PC Buyoff Failed,Machine Failure,UDR MF,Repair (UDR),UDR,Unscheduled Down (UD),UDT,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23222,WBA124,WBA124,461,2021-07-21 23:51:34,2021-07-21 23:53:00,86.0,3005,4008,5025,12382,SHORT TAIL,Machine Failure,UDR MF,Repair (UDR),UDR,Unscheduled Down (UD),UDT,S1941A,PG-DSO-20-88,1E115877B12,95706926,1601
23223,WBA124,WBA124,461,2021-07-21 23:53:00,2021-07-22 00:40:54,2874.0,3001,4001,5002,12373,Normal Production,Normal Production,PR N,Productive (PR),PR,Productive (PR),PRD,S1941A,PG-DSO-20-88,1E115877B12,95706926,1601
23224,WBA124,WBA124,461,2021-07-22 00:40:54,2021-07-22 00:45:38,284.0,3002,4002,5036,12390,Waiting For Response,Waiting for Response,SB WR,Standby (SB),SB,Standby (SB),SBY,S1941A,PG-DSO-20-88,1E115877B12,95706926,1601
23225,WBA124,WBA124,461,2021-07-22 00:45:38,2021-07-22 00:46:30,52.0,3002,4002,5036,12390,Waiting For Response,Waiting for Response,SB WR,Standby (SB),SB,Standby (SB),SBY,None,None,None,None,None


In [3]:
%store status

Stored 'status' (DataFrame)


In [4]:
## note the last 3 input parameters
## input_interval is the number of days between input window start and input window end
## time_window_step, make sure this can be divided with 24
## target_timewindow is the hour difference between target time start and target time end

import datetime as d


def datetime(input_startyear, input_startmonth, input_startday, input_endyear, input_endmonth, input_endday, input_interval,
             time_window_step, target_timewindow):
    
    start = d.datetime(input_startyear, input_startmonth, input_startday,0,0,0)
    end = d.datetime(input_endyear, input_endmonth, input_endday,0,0,0)
    delta = end - start
    eachday = int(24/time_window_step) #make sure time_window_step can be divided with 24
    rows = [] 
    startdate = start
    
    for i in range(delta.days+1): #this add e.g. 7 sections for 7 days
        for i in range(eachday): #this adds e.g. 3 sections per day if the time_window is 8 hours
            enddate = startdate + d.timedelta(days=input_interval)
            target_enddate = enddate + d.timedelta(hours = target_timewindow)
            rows.append([startdate, enddate, enddate, target_enddate])
            startdate += d.timedelta(hours=time_window_step)

    df = pd.DataFrame(rows, columns=['Input_Time_Window_Start', 'Input_Time_Window_End', 
                                         'Target_Time_Window_Start', 'Target_Time_Window_End'])
    return df 

In [5]:
# start can either be 'Input_Time_Window_Start' or 'Target_Time_Window_Start'
# end can either be 'Input_Time_Window_End' or 'Target_Time_Window_End'

def count_majordown(frame,start,end):
    no_major_down = [] 

    #This method is trying to tell from which row to which row in the status dataframe should I pay attention to when calculating UDT
    #After you extracted the 2 indexes, you can the calculate how many UDT are there between the 2 index

    for index, row in frame.iterrows():
        a = row[start]
        b = row[end]
        
        # for each row of data in frame, find the number of major down between the input start and end time
        start_interval = max(list(status.index[a >= status['TIMESTAMP_START']])) + 1 #status row index with timestamp closest to our 'Input_Time_Window_Start'
        end_interval = min(list(status.index[b <= status['TIMESTAMP_START']])) #status row index with timestamp_end closest to our 'Input_Time_Window_End'
        count = 0
        result = status.iloc[start_interval:end_interval]
        for i in range(len(result)):
            if result.iloc[i]['LEVEL3'] == 'UDT' and result.iloc[i]['DURATION'] > 3600: ######## change the criteria for major down here ############
                count += 1
        if count > 0:
            no_major_down.append(count)
        else: 
            no_major_down.append(0)

    frame['NoMajorDown'] = no_major_down
    return frame

In [6]:
# Import the EQ alarms data from V_TFM_ALARMHISOTRY table
cur_vis = connection.cursor()
cur_vis.execute("select * from V_TFM_ALARMHISOTRY t where t.eq_id in ('WBA124') and t.timestamp_end >= to_date('1/5/2020', 'dd/mm/yyyy')  and t.timestamp_start <= to_date('14/7/2021', 'dd/mm/yyyy') order by timestamp_start")

data2 = []

for row in cur_vis: 
    data2.append(row)
    
cur_vis.close()

alarm_data = pd.DataFrame(data2, columns = ['EQ_ID',
                                       'NAME',
                                       'ALARM_ID',
                                       'ALARM_TEXT',
                                       'ALARM_TYPE',
                                       'ALARM_CLASS',
                                       'TIMESTAMP_START',
                                       'TIMESTAMP_END',
                                       'LOT'])  
alarm_data

,EQ_ID,NAME,ALARM_ID,ALARM_TEXT,ALARM_TYPE,ALARM_CLASS,TIMESTAMP_START,TIMESTAMP_END,LOT
0,WBA124,WBA124,44,1st Bond Non Stick,2,unspecified,2021-01-28 10:52:05,2021-01-28 10:52:08,None
1,WBA124,WBA124,27,PR Lead Quality Rejected,2,Attention Flag,2021-01-28 10:52:55,2021-01-28 10:52:59,None
2,WBA124,WBA124,27,PR Lead Quality Rejected,2,Attention Flag,2021-01-28 10:53:08,2021-01-28 10:53:10,None
3,WBA124,WBA124,27,PR Lead Quality Rejected,2,Attention Flag,2021-01-28 10:53:44,2021-01-28 10:53:46,None
4,WBA124,WBA124,23,PR Lead Tol Failure,2,Attention Flag,2021-01-28 10:53:54,2021-01-28 10:53:58,None
...,...,...,...,...,...,...,...,...,...
12577,WBA124,WBA124,5,Indexer Position Error,2,Equipment Status warning,2021-07-13 08:18:58,2021-07-13 08:19:02,None
12578,WBA124,WBA124,5,Indexer Position Error,2,Equipment Status warning,2021-07-13 08:19:16,2021-07-13 08:19:21,None
12579,WBA124,WBA124,5,Indexer Position Error,2,Equipment Status warning,2021-07-13 08:19:49,2021-07-13 08:19:52,None
12580,WBA124,WBA124,2,Window Clamp Error,2,unspecified,2021-07-13 08:22:56,2021-07-13 08:22:59,None


In [7]:
#define whats a machine hang up (short time UDT...e.g. DURATION < 3600)

def small_hangups(frame,start,end,days_back):
    hangups = [] 

    #This method is trying to tell from which row to which row in the status dataframe should I pay attention to when calculating UDT
    #After you extracted the 2 indexes, you can the calculate how many UDT are there between the 2 index

    for index, row in frame.iterrows():
        a = row[start] + d.timedelta(days=days_back)
        b = row[end] 
        
        # for each row of data in frame, find the number of major down between the input start and end time
        start_interval = max(list(status.index[a >= status['TIMESTAMP_START']])) + 1 #status row index with timestamp closest to our 'Input_Time_Window_Start'
        end_interval = min(list(status.index[b <= status['TIMESTAMP_START']])) #status row index with timestamp_end closest to our 'Input_Time_Window_End'
        count = 0
        result = status.iloc[start_interval:end_interval]
        
        #add in machine hang 
        for i in range(len(result)):
            if result.iloc[i]['LEVEL3'] == 'UDT' and result.iloc[i]['DURATION'] < 300: ######## change the criteria for major down here ############
                count += 1
        if count > 0:
            hangups.append(count)
        else: 
            hangups.append(0)

    frame[f'Small_HangUp_Past{7-days_back}days'] = hangups
    return frame

In [8]:
import matplotlib.pyplot as plt

def plot_graph(result):

    #only plot rows with MajorDown not equals to 0 
    plot = result[result['NoMajorDown'] != 0]
    
    fig, axs = plt.subplots(2,figsize=(16,9))
        
    axs[0].plot(result['Input_Time_Window_End'], result['Small_HangUp_Past7days'], 'purple',
               result['Input_Time_Window_End'], result['Small_HangUp_Past5days'], 'red',
               result['Input_Time_Window_End'], result['Small_HangUp_Past3days'], 'orange')
    axs[0].legend(['Small Hangups over past 7 days', 'Small Hangups over past 5 days', 'Small Hangups over past 3 days'])
    

    #label the scatter points for better view
    for idx, row in plot.iterrows():
        axs[1].annotate(row['NoMajorDown'], (plot['Input_Time_Window_End'][idx], plot['NoMajorDown'][idx]), (plot['Input_Time_Window_End'][idx], plot['NoMajorDown'][idx]*1.2))

    axs[1].scatter(plot['Input_Time_Window_End'], plot['NoMajorDown'], s=200)
    axs[1].plot(result['Input_Time_Window_End'], result['Alarm2Count'], 'red',
                result['Input_Time_Window_End'], result['Alarm4Count'], 'orange',
                result['Input_Time_Window_End'], result['Alarm10Count'], 'green',
                result['Input_Time_Window_End'], result['Alarm3Count'], 'yellow')
    axs[1].set_title("Alarm counts over the past 7 days")
    axs[1].legend(['Alarm2', 'Alarm4', 'Alarm10', 'Alarm3'])


In [9]:
#create a helper function to search the occurence of certain id within a specific time frame

def search_table(table, column_name, alarm_id, timestamp_column, res, final_column_name):
    alarms = [] 
    data = table[table[column_name] == alarm_id].reset_index()
    
    for idx, row in res.iterrows():
        a = row['Input_Time_Window_Start']
        b = row['Input_Time_Window_End']
        
        #this try except logic is because sometimes even the timestamp in the first row of the data table bigger than 'Input Time Window Start'
        try:
            start_interval = max(list(data.index[a >= data[timestamp_column]])) + 1
        except:
            start_interval = 0
        
        try: 
            end_interval = min(list(data.index[b <= data[timestamp_column]])) #this one does not need to -1 because if we want row 5 to 7, we want to use 8-5 = 3rows
        except:
            end_interval = len(data)
            
        alarms.append(end_interval - start_interval)
    res[final_column_name] = alarms

In [10]:
# 1. change input timeframe with df1 = count_majordown(datetime(2021,3,15,2021,3,30,7,6,6),'Target_Time_Window_Start', 'Target_Time_Window_End')
# 2. add_alarm(<alarm_id>, df1)
# 3. small_hangups(df1, 'Input_Time_Window_Start', 'Input_Time_Window_End')
# 4. plot the result df to obtain the graph as shown above

df1 = count_majordown(datetime(2021,1,28,2021,7,14,7,6,6),'Target_Time_Window_Start', 'Target_Time_Window_End')
res2 = small_hangups(df1, 'Input_Time_Window_Start', 'Input_Time_Window_End', 0)
res2 = small_hangups(res2, 'Input_Time_Window_Start', 'Input_Time_Window_End', 2) #small hangups over (past 7-2 = 5 days)
res2 = small_hangups(res2, 'Input_Time_Window_Start', 'Input_Time_Window_End', 4) #small hangups over (past 7-4 = 3 days)

#proportions of small hang up which happen during the past 3, 5 and 7 days
days7 = res2['Small_HangUp_Past7days']
days5 = res2['Small_HangUp_Past5days']
days3 = res2['Small_HangUp_Past3days']
proportion_5_days = days3/days7
proportion_7_days = days5/days7
res2["Proportion of 3 days hangup over 7 days"] = proportion_5_days
res2["Proportion of 5 days hangup over 7 days"] = proportion_7_days

impt_alarms = [2, 4, 10, 3 , 5, 90, 19]
for ele in impt_alarms:
    try:
        search_table(alarm_data, "ALARM_ID", ele, "TIMESTAMP_START", res2, f'Alarm{ele}Count')
    except Exception as e:
        print(e)
        print(f'Alarm {ele} not found!')

#find out the occurence of level 6 HANG UP within each time period
search_table(status, "STATE_NAME", "HANG UP", "TIMESTAMP_START", res2, "HangUPCount")
res2

,Input_Time_Window_Start,Input_Time_Window_End,Target_Time_Window_Start,Target_Time_Window_End,NoMajorDown,Small_HangUp_Past7days,Small_HangUp_Past5days,Small_HangUp_Past3days,Proportion of 3 days hangup over 7 days,Proportion of 5 days hangup over 7 days,Alarm2Count,Alarm4Count,Alarm10Count,Alarm3Count,Alarm5Count,Alarm90Count,Alarm19Count,HangUPCount
0,2021-01-28 00:00:00,2021-02-04 00:00:00,2021-02-04 00:00:00,2021-02-04 06:00:00,0,331,276,113,0.341390,0.833837,0,0,4,0,2,0,0,0
1,2021-01-28 06:00:00,2021-02-04 06:00:00,2021-02-04 06:00:00,2021-02-04 12:00:00,0,331,267,91,0.274924,0.806647,0,0,4,0,2,0,0,0
2,2021-01-28 12:00:00,2021-02-04 12:00:00,2021-02-04 12:00:00,2021-02-04 18:00:00,0,333,252,77,0.231231,0.756757,0,0,4,0,1,0,0,0
3,2021-01-28 18:00:00,2021-02-04 18:00:00,2021-02-04 18:00:00,2021-02-05 00:00:00,0,344,253,78,0.226744,0.735465,0,0,2,0,0,0,0,0
4,2021-01-29 00:00:00,2021-02-05 00:00:00,2021-02-05 00:00:00,2021-02-05 06:00:00,0,352,247,77,0.218750,0.701705,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2021-07-13 18:00:00,2021-07-20 18:00:00,2021-07-20 18:00:00,2021-07-21 00:00:00,0,153,153,51,0.333333,1.000000,0,0,0,0,0,0,0,5
668,2021-07-14 00:00:00,2021-07-21 00:00:00,2021-07-21 00:00:00,2021-07-21 06:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5
669,2021-07-14 06:00:00,2021-07-21 06:00:00,2021-07-21 06:00:00,2021-07-21 12:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5
670,2021-07-14 12:00:00,2021-07-21 12:00:00,2021-07-21 12:00:00,2021-07-21 18:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5


In [13]:
import cx_Oracle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def get_eq():
    connection = cx_Oracle.connect('HARVESTING_BEAR_INDEX', 'P@ssword(May2018)', 'harvestingbearindex.sin.infineon.com')
    cursor = connection.cursor()
    statement = "select a.Eq_Name, a.site, a.segment, a.eqgroup, a.pilot_process_name from EQ_TFM_LIST a where eq_name = 'WBA124'"
    result = cursor.execute(statement)
    data = []
    for i in result:
        data.append(i)
    data = pd.DataFrame(data = data, columns = ['EQ', 'Site', 'Segment', 'Eqpt Group', 'Process'])
    return data

def connect_site(site):
    db_info = {'SIN': {'TNS': 'OME.SIN.INFINEON.COM', 'username': 'tfm_reporting', 'PW': 'reporting'},
     'BAT': {'TNS': 'OME.BTH.INFINEON.COM', 'username': 'TFM4CEBERUS', 'PW': 'TFM4CEBERUS'},
     'MAL': {'TNS': 'OME.MKZ.INFINEON.COM', 'username': 'tfm_reporting', 'PW': 'tfm_reporting123'},
     'WUX': {'TNS': 'OME.WUX.INFINEON.COM', 'username': 'tfm_reporting', 'PW': 'TFM_REPORTING'},
     'RBG': {'TNS': 'OME.RBG.INFINEON.COM', 'username': 'Tfm_reporting', 'PW': 'new_20140415_TFM_REPO'},
     'WAR': {'TNS': 'OME.WAR.INFINEON.COM', 'username': 'TFM_REPORTING', 'PW': 'LDTSP8M(:M=U<N<?eYu)'},
     'CEG': {'TNS': 'TFMC.CEG.INFINEON.COM', 'username': 'tfm_reporting', 'PW': 'r3porting@CEG(247)'},
     'CHE': {'TNS': 'OME.CJJ.INFINEON.COM', 'username': 'tfm_reporting', 'PW': 'r3porting@CJJ(246)'},
     'TIJ': {'TNS': 'OME.TIJ.INFINEON.COM', 'username': 'TFM_REPORTING', 'PW': '24AM:!.W#:<uaZ;2wwCY'},
     'HAR': {'TNS': 'harvestingbearindex.sin.infineon.com', 'username': 'HARVESTING_BEAR_INDEX', 'PW': 'P@ssword(May2018)'}}
    
    site_info = db_info[site]
    connection = cx_Oracle.connect(site_info['username'], site_info['PW'], site_info['TNS'])
    #cursor = connection.cursor()
    return connection

def get_eq_data(eq, time_start, time_end, cursor):
    statement = "select t.eq_id, t.timestamp_start, t.timestamp_end, t.Duration/60 as duration, t.Level5_Name from V_TFM_RAW_DATA_METAKEY_ITFM t where t.eq_id = :1 and t.timestamp_start >= to_date(:2, 'yyyy-mm-dd hh24:mi:ss') and t.timestamp_start < to_date(:3, 'yyyy-mm-dd hh24:mi:ss') order by t.timestamp_start asc"
    param = (eq, time_start, time_end)
    result = cursor.execute(statement, param)
    data = []
    for i in result:
        data.append(i)
    data = pd.DataFrame(data = data, columns = ['EQ', 'Timestamp_start', 'Timestamp_end', 'Duration', 'Level5_Name'])
    return data

def combine_same_status(data):
    data['Level5_Name_Last'] = data['Level5_Name'].shift(1)
    data['Compare'] = [data['Level5_Name_Last'][r] == data['Level5_Name'][r] for r in range(len(data))]
    data['Case'] = 1
    
    for i in range(1, len(data)):
        if data['Compare'][i] == True:
            data['Case'][i] = data['Case'][i-1]
        else:
            data['Case'][i] = data['Case'][i-1]+1
    
    output = data.groupby(['EQ', 'Case', 'Level5_Name']).agg({'Timestamp_start': 'min', 'Timestamp_end': 'max', 'Duration': 'sum'})
    output = output.reset_index()
    return output

def assign_scenario(output):
    step1_dict = {'Machine Failure': {'Code': 'R', 'Cutoff': 6},
              'Change Lot/Setup': {'Code': 'S', 'Cutoff': 15},
              'Change Setup': {'Code': 'S', 'Cutoff': 15},
              'Setup (SDS)': {'Code': 'S', 'Cutoff': 15},
              'Planned Offline Maintenance': {'Code': 'PM', 'Cutoff': 60},
              'Maintenance (SDM)': {'Code': 'PM', 'Cutoff': 60},
              'Maintenance': {'Code': 'PM', 'Cutoff': 60},
              'Normal Production': {'Code': 'PR', 'Cutoff': 10},
              'Productive (PR)': {'Code': 'PR', 'Cutoff': 10}}
    
    for k in step1_dict.keys():
        output.loc[output['Level5_Name']==k, 'Code'] = step1_dict[k]['Code']
        output.loc[((output['Level5_Name']==k) & (output['Duration']>=step1_dict[k]['Cutoff'])), 'Status'] = 'Accept'
        output.loc[((output['Level5_Name']==k) & (output['Duration']<step1_dict[k]['Cutoff'])), 'Status'] = 'Reject'
    return output

def get_RSPM_accept(output):
    table = output.copy()
    table = table[(table['Code'].isin(['R', 'S', 'PM'])) & (table['Status'] == 'Accept')]
    table['IsStart'] = None
    table['IsEnd'] = None
    table['Result'] = None
    table = table.reset_index()
    del table['index']
    return table

def assign_startend_status(table, output):
    if len(table) > 0:
        Endtime_dict = {'R': 1, 'S': 1, 'PM': 3}
        for k in Endtime_dict.keys():
            table.loc[table['Code'] == k, 'Endtime'] = table.loc[table['Code'] == k,'Timestamp_end'] + pd.Timedelta(days = Endtime_dict[k])
        current_case = table['Code'][0]
        position = table['Case'][0]
        end_time = table['Endtime'][0]
        table['IsStart'][0] = table['Code'][0] + '-' + 'Start'
            
        for i in range(1, len(table)):
            if table['Timestamp_start'][i] >= end_time:
                table['IsEnd'][i-1] = table['Code'][i-1] + '-' +'End'
                table['Result'][i-1] = 'Pass'
                table['IsStart'][i] = table['Code'][i] + '-' + 'Start'
                current_case = table['Code'][i]
                position = table['Case'][i]
                end_time = table['Endtime'][i]
            else:
                if table['Code'][i] != 'R':
                    table['IsEnd'][i-1] = table['Code'][i-1] + '-' + 'End'
                    table['Result'][i-1] = 'Reset'
                    table['IsStart'][i] = table['Code'][i] + '-' + 'Start'
                    current_case = table['Code'][i]
                    position = table['Case'][i]
                    end_time = table['Endtime'][i]
                elif table['Code'][i-1] != 'R':
                    table['IsEnd'][i-1] = table['Code'][i-1] + '-' + 'End'
                    table['Result'][i-1] = 'Fail'
                    table['IsStart'][i] = table['Code'][i] + '-' + 'Start'
                    current_case = table['Code'][i]
                    position = table['Case'][i]
                    end_time = table['Endtime'][i]
                else:
                    window_start = position
                    window_end = table['Case'][i]
                    output_window = output[(output['Case'] >= window_start) & (output['Case'] <= window_end)].copy()
                    output_window['Code_Status'] = output_window['Code'] + output_window['Status']
                    if 'PRAccept' in list(output_window['Code_Status']):
                        table['IsEnd'][i-1] = table['Code'][i-1] + '-' + 'End'
                        table['Result'][i-1] = 'Fail'
                        table['IsStart'][i] = table['Code'][i] + '-' + 'Start'
                        current_case = table['Code'][i]
                        position = table['Case'][i]
                        end_time = table['Endtime'][i]
                    else:
                        current_case = table['Code'][i]
                        position = table['Case'][i]
                        end_time = table['Endtime'][i]
    return table

eq_info = get_eq()
site_lst = list(eq_info['Site'].unique())
site_eq_dict = {}
for i in site_lst:
    site_eq_dict.update({i: list(eq_info.loc[eq_info['Site'] == i, 'EQ'])})
    print(i)

changesetup = []
machinefailure = [] 
changesetup_pass = [] 
changesetup_fail = []
changesetup_reset = [] 
machinefailure_pass = [] 
machinefailure_fail = [] 
machinefailure_reset = [] 

for k in range(len(res2)):
    time_start = str(res2.iloc[k]['Input_Time_Window_Start'])
    time_end = str(res2.iloc[k]['Input_Time_Window_End'])

    table_lst = [] #compute the pass, fail reset status for one row of dataframe
    for j in site_eq_dict.keys():
        cursor = connect_site(j).cursor()
        for i in site_eq_dict[j]:
            data = get_eq_data(i,time_start,time_end, cursor)
            if len(data) > 0:
                output = combine_same_status(data)
                output = assign_scenario(output)
                table_temp = get_RSPM_accept(output)
                if len(table_temp) > 0:
                    table_temp = assign_startend_status(table_temp, output)
                    table_temp = table_temp.loc[table_temp['Timestamp_start'] < time_end]
                    del table_temp['Endtime']
                    del table_temp['Case']
                    table_temp['Situation'] = 1
                    for m in range(1, len(table_temp)):
                        if table_temp['IsStart'][m] == None:
                            table_temp['Situation'][m] = table_temp['Situation'][m-1]
                        else:
                            table_temp['Situation'][m] = table_temp['Situation'][m-1] + 1
                    table_temp['IsStart'].fillna(method = 'ffill', inplace = True)
                    table_temp['IsEnd'].fillna(method = 'bfill', inplace = True)
                    table_temp['Result'].fillna(method = 'bfill', inplace = True)
                    table_final = table_temp.groupby(['Situation', 'EQ', 'Level5_Name', 'Code', 'Status', 'IsStart', 'IsEnd', 'Result']).agg({'Timestamp_start': 'min', 'Timestamp_end': 'max', 'Duration': 'sum'})
                    table_final = table_final.reset_index()
                    del table_final['Situation']
                    table_final['Site'] = j
                    table_lst.append(table_final)


    pivot1 = table_final["Level5_Name"].value_counts()

    try:
        machinefailure.append(pivot1.loc["Machine Failure"])
    except:
        machinefailure.append(0)
    try:
        changesetup.append(pivot1.loc["Change Setup"])
    except:
        changesetup.append(0)
            
    pivot2 = table_final.pivot_table(index=['Level5_Name'], columns=['Result'], aggfunc='count', values=['EQ'])
    try:
        changesetup_fail.append(int(pivot2.loc["Change Setup", "EQ"]["Fail"]))
    except:
        changesetup_fail.append(0)
    try:
        changesetup_pass.append(int(pivot2.loc["Change Setup", "EQ"]["Pass"]))
    except:
        changesetup_pass.append(0)
    try:
        changesetup_reset.append(int(pivot2.loc["Change Setup", "EQ"]["Reset"]))
    except:
        changesetup_reset.append(0)
        
    try:
        machinefailure_fail.append(int(pivot2.loc["Machine Failure", "EQ"]["Fail"]))
    except:
        machinefailure_fail.append(0)
    try:
        machinefailure_pass.append(int(pivot2.loc["Machine Failure", "EQ"]["Pass"]))
    except:
        machinefailure_pass.append(0)
    try:
        machinefailure_reset.append(int(pivot2.loc["Machine Failure", "EQ"]["Reset"]))
    except:
        machinefailure_reset.append(0)

res2["Machine Failure"] = machinefailure
res2["Machine Failure Passed"] = machinefailure_pass
res2["Machine Failure Failed"] = machinefailure_fail
res2["Machine Failure Reset"] = machinefailure_reset
res2["Change Setup"] = changesetup
res2["Change Setup Passed"] = changesetup_pass
res2["Change Setup Failed"] = changesetup_fail
res2["Change Setup Reset"] = changesetup_reset
print('Finish Data Calculation')

BAT
Finish Data Calculation


In [14]:
#days since last maintenance 
#transaction error count for each special transaction error
connection = cx_Oracle.connect('bth_odsprod', 'bth_odsprodbth', 'ODSPROD.BTH.INFINEON.COM')
cur_vis = connection.cursor()
statement = """select a.ifxequipmentname, a.ifxtxndate, a.ifxerrorid, a.ifxtransactioname, a.ifxerrormessage from a_ifxtxnerrorlog a 
                where a.ifxequipmentname in ('WBA124') and a.ifxtxndate >= to_date('1/1/2021', 'dd/mm/yyyy') 
                and a.ifxtxndate < to_date('14/7/2021', 'dd/mm/yyyy')
                order by ifxtxndate"""
cur_vis.execute(statement)

data = []

for row in cur_vis: 
    data.append(row)
    
xsaction_error = pd.DataFrame(data, columns = ['IFXEQUIPMENTNAME',
                                   'IFXTXNDATE',
                                   'IFXERRORID',
                                   'IFXTRANSACTIONAME',
                                   'IFXERRORMESSAGE'])
xsaction_error

,IFXEQUIPMENTNAME,IFXTXNDATE,IFXERRORID,IFXTRANSACTIONAME,IFXERRORMESSAGE
0,WBA124,2021-01-01 00:54:30,1087242244,PrepareJob,Camstar.ValidateLot: ifxTrackInLot_E0005: Reso...
1,WBA124,2021-01-01 00:54:30,1087242244,TrackInLot,ifxTrackInLot_E0005: Resource WBA124 membutuhk...
2,WBA124,2021-01-01 00:55:19,1087242244,PrepareJob,Camstar.ValidateLot: ifxTrackInLot_E0005: Reso...
3,WBA124,2021-01-01 00:55:19,1087242244,TrackInLot,ifxTrackInLot_E0005: Resource WBA124 membutuhk...
4,WBA124,2021-01-01 06:11:24,1087242244,TrackInLot,ifxTrackInLot_E0005: Resource WBA124 membutuhk...
...,...,...,...,...,...
309,WBA124,2021-07-07 21:53:07,1087242244,EquipmentMaterialsSetup,EquipmentMaterialsSetup_E0001: There are no ma...
310,WBA124,2021-07-09 16:49:35,1087242244,AssemblyMotherLotWIPMain,WIPMain_E0027: 1E113455B10 is on hold (with di...
311,WBA124,2021-07-09 21:13:44,1087242244,AssemblyMotherLotWIPMain,WIPMain_E0027: 1E113455B10 is on hold (with di...
312,WBA124,2021-07-10 21:03:59,1087242244,PrepareJob,Camstar.ValidateLot: ifxTrackInLot_E0001: WBA1...


In [15]:
xsaction_error_id = xsaction_error.IFXERRORID.unique() #there are only 2 kinds of error id
for ele in xsaction_error_id:
    search_table(xsaction_error, "IFXERRORID", ele, "IFXTXNDATE", res2, f"xscation error {ele} Count")
res2

,Input_Time_Window_Start,Input_Time_Window_End,Target_Time_Window_Start,Target_Time_Window_End,NoMajorDown,Small_HangUp_Past7days,Small_HangUp_Past5days,Small_HangUp_Past3days,Proportion of 3 days hangup over 7 days,Proportion of 5 days hangup over 7 days,Alarm2Count,Alarm4Count,Alarm10Count,Alarm3Count,Alarm5Count,Alarm90Count,Alarm19Count,HangUPCount,Machine Failure,Machine Failure Passed,Machine Failure Failed,Machine Failure Reset,Change Setup,Change Setup Passed,Change Setup Failed,Change Setup Reset,xscation error 1087242244 Count,xscation error 13500912 Count,xscation error 13500638 Count,xscation error 999 Count
0,2021-01-28 00:00:00,2021-02-04 00:00:00,2021-02-04 00:00:00,2021-02-04 06:00:00,0,331,276,113,0.341390,0.833837,0,0,4,0,2,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0
1,2021-01-28 06:00:00,2021-02-04 06:00:00,2021-02-04 06:00:00,2021-02-04 12:00:00,0,331,267,91,0.274924,0.806647,0,0,4,0,2,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0
2,2021-01-28 12:00:00,2021-02-04 12:00:00,2021-02-04 12:00:00,2021-02-04 18:00:00,0,333,252,77,0.231231,0.756757,0,0,4,0,1,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0
3,2021-01-28 18:00:00,2021-02-04 18:00:00,2021-02-04 18:00:00,2021-02-05 00:00:00,0,344,253,78,0.226744,0.735465,0,0,2,0,0,0,0,0,19,0,15,4,6,1,4,1,6,0,0,0
4,2021-01-29 00:00:00,2021-02-05 00:00:00,2021-02-05 00:00:00,2021-02-05 06:00:00,0,352,247,77,0.218750,0.701705,0,0,2,0,0,0,0,0,19,0,15,4,6,1,4,1,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2021-07-13 18:00:00,2021-07-20 18:00:00,2021-07-20 18:00:00,2021-07-21 00:00:00,0,153,153,51,0.333333,1.000000,0,0,0,0,0,0,0,5,6,2,4,0,1,0,1,0,0,0,0,0
668,2021-07-14 00:00:00,2021-07-21 00:00:00,2021-07-21 00:00:00,2021-07-21 06:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,6,2,4,0,1,0,1,0,0,0,0,0
669,2021-07-14 06:00:00,2021-07-21 06:00:00,2021-07-21 06:00:00,2021-07-21 12:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,5,1,4,0,1,0,1,0,0,0,0,0
670,2021-07-14 12:00:00,2021-07-21 12:00:00,2021-07-21 12:00:00,2021-07-21 18:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,5,1,4,0,1,0,1,0,0,0,0,0


In [16]:
#day from maintenance
sql_statement = """select mrb.maintenancereqname, r.resourcename, mr.scheduledate, ms.maintcompletiondate, cs.lastchangedate from maintenancereqbase mrb
                    inner join maintenancereq mr on mrb.maintenancereqbaseid = mr.maintenancereqbaseid
                    inner join assignedmaintreq amr on mr.maintenancereqid = amr.maintenancereqid
                    inner join resourcedef r on r.resourceid = amr.parentid
                    inner join maintenancestatus ms on ms.assignedmaintreqid = amr.assignedmaintreqid
                    inner join changestatus cs on mr.changehistoryid = cs.changestatusid
                    where r.resourcename in ('WBA124')
                    and mrb.maintenancereqname like '043%'
                    order by LASTCHANGEDATE
                    """
cur_vis.execute(sql_statement)

data = []

for row in cur_vis: 
    data.append(row)
    
cur_vis.close()

maintenance_information = pd.DataFrame(data, columns = ['MAINTENANCEREQNAME',
                                   'RESOURCENAME',
                                   'SCHEDULEDATE',
                                   'MAINTCOMPLETIONDATE',
                                   'LASTCHANGEDATE'])
maintenance_information

,MAINTENANCEREQNAME,RESOURCENAME,SCHEDULEDATE,MAINTCOMPLETIONDATE,LASTCHANGEDATE
0,043000001623,WBA124,2019-11-13,2019-11-12 00:00:00,2019-11-12 09:59:58
1,043000006732,WBA124,2020-05-13,2020-05-13 07:52:00,2020-05-13 07:52:00
2,043000011783,WBA124,2020-11-13,2020-11-12 08:11:16,2020-11-12 08:11:16
3,043000015782,WBA124,2021-05-13,2021-05-11 00:00:00,2021-05-11 15:06:17


In [17]:
days_since_last_maintenance = [] 
for idx, row in res2.iterrows():
    a = row['Input_Time_Window_Start']
    index = np.where(list(a>maintenance_information["LASTCHANGEDATE"]))[0]

    date = maintenance_information.iloc[max(index)]['LASTCHANGEDATE']
    days = a - date
    days_since_last_maintenance.append(days.days)
res2["Days since last maintenance"] = days_since_last_maintenance
res2

,Input_Time_Window_Start,Input_Time_Window_End,Target_Time_Window_Start,Target_Time_Window_End,NoMajorDown,Small_HangUp_Past7days,Small_HangUp_Past5days,Small_HangUp_Past3days,Proportion of 3 days hangup over 7 days,Proportion of 5 days hangup over 7 days,Alarm2Count,Alarm4Count,Alarm10Count,Alarm3Count,Alarm5Count,Alarm90Count,Alarm19Count,HangUPCount,Machine Failure,Machine Failure Passed,Machine Failure Failed,Machine Failure Reset,Change Setup,Change Setup Passed,Change Setup Failed,Change Setup Reset,xscation error 1087242244 Count,xscation error 13500912 Count,xscation error 13500638 Count,xscation error 999 Count,Days since last maintenance
0,2021-01-28 00:00:00,2021-02-04 00:00:00,2021-02-04 00:00:00,2021-02-04 06:00:00,0,331,276,113,0.341390,0.833837,0,0,4,0,2,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0,76
1,2021-01-28 06:00:00,2021-02-04 06:00:00,2021-02-04 06:00:00,2021-02-04 12:00:00,0,331,267,91,0.274924,0.806647,0,0,4,0,2,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0,76
2,2021-01-28 12:00:00,2021-02-04 12:00:00,2021-02-04 12:00:00,2021-02-04 18:00:00,0,333,252,77,0.231231,0.756757,0,0,4,0,1,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0,77
3,2021-01-28 18:00:00,2021-02-04 18:00:00,2021-02-04 18:00:00,2021-02-05 00:00:00,0,344,253,78,0.226744,0.735465,0,0,2,0,0,0,0,0,19,0,15,4,6,1,4,1,6,0,0,0,77
4,2021-01-29 00:00:00,2021-02-05 00:00:00,2021-02-05 00:00:00,2021-02-05 06:00:00,0,352,247,77,0.218750,0.701705,0,0,2,0,0,0,0,0,19,0,15,4,6,1,4,1,6,0,0,0,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2021-07-13 18:00:00,2021-07-20 18:00:00,2021-07-20 18:00:00,2021-07-21 00:00:00,0,153,153,51,0.333333,1.000000,0,0,0,0,0,0,0,5,6,2,4,0,1,0,1,0,0,0,0,0,63
668,2021-07-14 00:00:00,2021-07-21 00:00:00,2021-07-21 00:00:00,2021-07-21 06:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,6,2,4,0,1,0,1,0,0,0,0,0,63
669,2021-07-14 06:00:00,2021-07-21 06:00:00,2021-07-21 06:00:00,2021-07-21 12:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,5,1,4,0,1,0,1,0,0,0,0,0,63
670,2021-07-14 12:00:00,2021-07-21 12:00:00,2021-07-21 12:00:00,2021-07-21 18:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,5,1,4,0,1,0,1,0,0,0,0,0,63


# Create helper function which breaks down each row into 6 hours interval

In [18]:
import datetime
test_dataset = res2.copy()

#create input_time_window_start and input_time_window_end by row
def create_datetime(dataframe, row, hours):
    df = pd.DataFrame(columns=['Input_Time_Window_Start', 'Input_Time_Window_End'])
    input_start = dataframe.iloc[row]['Input_Time_Window_Start']
    counter = 0 
    while input_start < dataframe.iloc[row]['Input_Time_Window_End']:
        input_end = input_start + datetime.timedelta(hours = hours)
        df = df.append({'Input_Time_Window_Start': input_start, 'Input_Time_Window_End': input_end}, ignore_index=True)
        input_start += datetime.timedelta(hours = hours)
        counter += 1
    return df

In [19]:
#count number of hang ups within these time period
def small_hangups(frame,start,end):
    hangups = [] 

    #This method is trying to tell from which row to which row in the status dataframe should I pay attention to when calculating UDT
    #After you extracted the 2 indexes, you can the calculate how many UDT are there between the 2 index

    for index, row in frame.iterrows():
        a = row[start]
        b = row[end] 
        
        # for each row of data in frame, find the number of major down between the input start and end time
        start_interval = max(list(status.index[a >= status['TIMESTAMP_START']])) + 1 #status row index with timestamp closest to our 'Input_Time_Window_Start'
        end_interval = min(list(status.index[b <= status['TIMESTAMP_START']])) #status row index with timestamp_end closest to our 'Input_Time_Window_End'
        count = 0
        result = status.iloc[start_interval:end_interval]
        
        #add in machine hang 
        for i in range(len(result)):
            if result.iloc[i]['LEVEL3'] == 'UDT' and result.iloc[i]['DURATION'] < 300: ######## change the criteria for major down here ############
                count += 1
        if count > 0:
            hangups.append(count)
        else: 
            hangups.append(0)

    frame['No. Small hangups'] = hangups
    return frame

In [20]:
def EEEC_stats(res2, row, new_datetime):
    time_start = str(res2.iloc[k]['Input_Time_Window_Start'])
    time_end = str(res2.iloc[k]['Input_Time_Window_End'])
    machinefailure = [] 
    changesetup = []
    machine_pass = []
    machine_fail = []
    machine_reset = []
    setup_pass = []
    setup_fail = []
    setup_reset = []

    table_lst = [] #compute the pass, fail reset status for one row of dataframe
    for j in site_eq_dict.keys():
        cursor = connect_site(j).cursor()
        for i in site_eq_dict[j]:
            data = get_eq_data(i,time_start,time_end, cursor)
            if len(data) > 0:
                output = combine_same_status(data)
                output = assign_scenario(output)
                table_temp = get_RSPM_accept(output)
                if len(table_temp) > 0:
                    table_temp = assign_startend_status(table_temp, output)
                    table_temp = table_temp.loc[table_temp['Timestamp_start'] < time_end]
                    del table_temp['Endtime']
                    del table_temp['Case']
                    table_temp['Situation'] = 1
                    for m in range(1, len(table_temp)):
                        if table_temp['IsStart'][m] == None:
                            table_temp['Situation'][m] = table_temp['Situation'][m-1]
                        else:
                            table_temp['Situation'][m] = table_temp['Situation'][m-1] + 1
                    table_temp['IsStart'].fillna(method = 'ffill', inplace = True)
                    table_temp['IsEnd'].fillna(method = 'bfill', inplace = True)
                    table_temp['Result'].fillna(method = 'bfill', inplace = True)
                    table_final = table_temp.groupby(['Situation', 'EQ', 'Level5_Name', 'Code', 'Status', 'IsStart', 'IsEnd', 'Result']).agg({'Timestamp_start': 'min', 'Timestamp_end': 'max', 'Duration': 'sum'})
                    table_final = table_final.reset_index()
                    del table_final['Situation']
                    table_final['Site'] = j
                    table_lst.append(table_final)
                    
    for i in range(len(new_datetime)):
        filtered = table_final[(table_final['Timestamp_start'] > new_datetime.iloc[i]['Input_Time_Window_Start']) &(table_final['Timestamp_start'] < new_datetime.iloc[i]['Input_Time_Window_End'])]

        #count by failure
        count_by_failure = filtered['Level5_Name'].value_counts()
        try:
            machinefailure.append(count_by_failure['Machine Failure'])
        except:
            machinefailure.append(0)
        try:
            changesetup.append(count_by_failure['Change Setup']) 
        except:
            changesetup.append(0) 

        #count by status
        count_by_status = filtered[['Level5_Name', 'Result']].value_counts()
        try:
            machine_pass.append(count_by_status['Machine Failure', 'Pass'])
        except:
            machine_pass.append(0)
        try:
            machine_fail.append(count_by_status['Machine Failure', 'Fail'])
        except:
            machine_fail.append(0)
        try:
            machine_reset.append(count_by_status['Machine Failure', 'Reset'])
        except:
            machine_reset.append(0)
        try:
            setup_pass.append(count_by_status['Change Setup', 'Pass'])
        except:
            setup_pass.append(0)
        try:
            setup_fail.append(count_by_status['Change Setup', 'Fail'])
        except:
            setup_fail.append(0)
        try:
            setup_reset.append(count_by_status['Change Setup', 'Reset'])
        except:
            setup_reset.append(0)
            
    new_datetime["Machine Failure"] = machinefailure
    new_datetime["Machine Failure Passed"] = machine_pass
    new_datetime["Machine Failure Failed"] = machine_fail
    new_datetime["Machine Failure Reset"] = machine_reset
    new_datetime["Change Setup"] = changesetup
    new_datetime["Change Setup Passed"] = setup_pass
    new_datetime["Change Setup Failed"] = setup_fail
    new_datetime["Change Setup Reset"] = setup_reset
    return new_datetime

In [21]:
### concat the results together ### each row can be broken down into 28 rows
training_data_len = round(0.75*len(res2)) #90
validation_data_len = round(0.2*len(res2)) #24
test_data_len = len(res2) - training_data_len - validation_data_len #6

train_dataset = res2[:training_data_len]
validation_dataset = res2[training_data_len:training_data_len+validation_data_len].reset_index()
test_dataset = res2[-test_data_len:].reset_index()

In [22]:
def breakdown_into_row(dataset):
    result = pd.DataFrame()
    impt_alarms = [2, 4, 10, 3 , 5, 90, 19]
    xsaction_error_id = xsaction_error.IFXERRORID.unique()
    for i in range(len(dataset)): 
        new_datetime = create_datetime(dataset, i, 6) #creates a row breakdown for every rows in dataset into 6 hours interval
        new_datetime = small_hangups(new_datetime, 'Input_Time_Window_Start', 'Input_Time_Window_End')
        
        for ele in impt_alarms:
            try:
                search_table(alarm_data, "ALARM_ID", ele, "TIMESTAMP_START", new_datetime, f'Alarm{ele}Count')
            except Exception as e:
                print(e)
                print(f'Alarm {ele} not found!')
        search_table(status, "STATE_NAME", "HANG UP", "TIMESTAMP_START", new_datetime, "HangUPCount")
        new_datetime = EEEC_stats(res2, 0, new_datetime)
        for ele in xsaction_error_id:
            search_table(xsaction_error, "IFXERRORID", ele, "IFXTXNDATE", new_datetime, f"xscation error {ele} Count")
        result = pd.concat([result, new_datetime], ignore_index=True)
    return result

In [23]:
train_dataset_by_row = breakdown_into_row(train_dataset)
validation_dataset_by_row = breakdown_into_row(validation_dataset)
test_dataset_by_row = breakdown_into_row(test_dataset)

In [24]:
len(train_dataset_by_row), len(validation_dataset_by_row), len(test_dataset_by_row)

(14112, 3752, 952)

In [25]:
%store train_dataset_by_row validation_dataset_by_row test_dataset_by_row res2 

Stored 'train_dataset_by_row' (DataFrame)
Stored 'validation_dataset_by_row' (DataFrame)
Stored 'test_dataset_by_row' (DataFrame)
Stored 'res2' (DataFrame)


In [26]:
res2.columns

Index(['Input_Time_Window_Start', 'Input_Time_Window_End',
       'Target_Time_Window_Start', 'Target_Time_Window_End', 'NoMajorDown',
       'Small_HangUp_Past7days', 'Small_HangUp_Past5days',
       'Small_HangUp_Past3days', 'Proportion of 3 days hangup over 7 days',
       'Proportion of 5 days hangup over 7 days', 'Alarm2Count', 'Alarm4Count',
       'Alarm10Count', 'Alarm3Count', 'Alarm5Count', 'Alarm90Count',
       'Alarm19Count', 'HangUPCount', 'Machine Failure',
       'Machine Failure Passed', 'Machine Failure Failed',
       'Machine Failure Reset', 'Change Setup', 'Change Setup Passed',
       'Change Setup Failed', 'Change Setup Reset',
       'xscation error 1087242244 Count', 'xscation error 13500912 Count',
       'xscation error 13500638 Count', 'xscation error 999 Count',
       'Days since last maintenance'],
      dtype='object')

In [27]:
train_dataset_by_row.columns

Index(['Input_Time_Window_Start', 'Input_Time_Window_End', 'No. Small hangups',
       'Alarm2Count', 'Alarm4Count', 'Alarm10Count', 'Alarm3Count',
       'Alarm5Count', 'Alarm90Count', 'Alarm19Count', 'HangUPCount',
       'Machine Failure', 'Machine Failure Passed', 'Machine Failure Failed',
       'Machine Failure Reset', 'Change Setup', 'Change Setup Passed',
       'Change Setup Failed', 'Change Setup Reset',
       'xscation error 1087242244 Count', 'xscation error 13500912 Count',
       'xscation error 13500638 Count', 'xscation error 999 Count'],
      dtype='object')

In [28]:
res2

,Input_Time_Window_Start,Input_Time_Window_End,Target_Time_Window_Start,Target_Time_Window_End,NoMajorDown,Small_HangUp_Past7days,Small_HangUp_Past5days,Small_HangUp_Past3days,Proportion of 3 days hangup over 7 days,Proportion of 5 days hangup over 7 days,Alarm2Count,Alarm4Count,Alarm10Count,Alarm3Count,Alarm5Count,Alarm90Count,Alarm19Count,HangUPCount,Machine Failure,Machine Failure Passed,Machine Failure Failed,Machine Failure Reset,Change Setup,Change Setup Passed,Change Setup Failed,Change Setup Reset,xscation error 1087242244 Count,xscation error 13500912 Count,xscation error 13500638 Count,xscation error 999 Count,Days since last maintenance
0,2021-01-28 00:00:00,2021-02-04 00:00:00,2021-02-04 00:00:00,2021-02-04 06:00:00,0,331,276,113,0.341390,0.833837,0,0,4,0,2,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0,76
1,2021-01-28 06:00:00,2021-02-04 06:00:00,2021-02-04 06:00:00,2021-02-04 12:00:00,0,331,267,91,0.274924,0.806647,0,0,4,0,2,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0,76
2,2021-01-28 12:00:00,2021-02-04 12:00:00,2021-02-04 12:00:00,2021-02-04 18:00:00,0,333,252,77,0.231231,0.756757,0,0,4,0,1,0,0,0,19,0,15,4,5,0,4,1,5,0,0,0,77
3,2021-01-28 18:00:00,2021-02-04 18:00:00,2021-02-04 18:00:00,2021-02-05 00:00:00,0,344,253,78,0.226744,0.735465,0,0,2,0,0,0,0,0,19,0,15,4,6,1,4,1,6,0,0,0,77
4,2021-01-29 00:00:00,2021-02-05 00:00:00,2021-02-05 00:00:00,2021-02-05 06:00:00,0,352,247,77,0.218750,0.701705,0,0,2,0,0,0,0,0,19,0,15,4,6,1,4,1,6,0,0,0,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2021-07-13 18:00:00,2021-07-20 18:00:00,2021-07-20 18:00:00,2021-07-21 00:00:00,0,153,153,51,0.333333,1.000000,0,0,0,0,0,0,0,5,6,2,4,0,1,0,1,0,0,0,0,0,63
668,2021-07-14 00:00:00,2021-07-21 00:00:00,2021-07-21 00:00:00,2021-07-21 06:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,6,2,4,0,1,0,1,0,0,0,0,0,63
669,2021-07-14 06:00:00,2021-07-21 06:00:00,2021-07-21 06:00:00,2021-07-21 12:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,5,1,4,0,1,0,1,0,0,0,0,0,63
670,2021-07-14 12:00:00,2021-07-21 12:00:00,2021-07-21 12:00:00,2021-07-21 18:00:00,0,153,153,32,0.209150,1.000000,0,0,0,0,0,0,0,5,5,1,4,0,1,0,1,0,0,0,0,0,63
